In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import Table, hstack, vstack, join
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from spectral_cube import SpectralCube
import scipy 
from reproject import reproject_interp

import astropy.constants as ac
import astropy.units as au
from astropy import stats
from astrodendro.analysis import PPStatistic
from astrodendro import Dendrogram, pp_catalog
from astropy.wcs import WCS
from collections import Counter

from astropy.io import fits
import matplotlib as mpl
import pyregion
import aplpy
import math
import os
import pickle
from tqdm.auto import tqdm

from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D

from analysis_phangs_hst import dendro_dendro, dendro_misc

In [2]:
def get_trunkindexmap(dendro):
    """Get index map of trunk idx"""
    
    # print('[INFO] Getting indexmap of dendrogram trunks')
    mask = 0
    for i in range(len(dendro.trunk)):

        trunk = dendro.trunk[i]
        idx = trunk.idx
        mask = trunk.get_mask()
        mask = mask*1.0
        mask[mask==0] = np.nan
        mask[mask==1] = idx

        if i==0: 
            indexmap_trunk = mask
        else: 
            indexmap_trunk = np.nanmean(np.array([indexmap_trunk,mask]), axis=0)  

    indexmap_trunk[np.isnan(indexmap_trunk)] = -1
    
    return(indexmap_trunk)


def get_circrad(data, pixsize):
    """Returns circularised radius in arcsec"""
    area = np.nansum((data > -1)*1)
    radius = (area/np.pi)**0.5
    radius = radius * pixsize.to('arcsec')
    return(radius)

In [3]:
# Get Galaxy properties
galaxy = 'ngc0628'
sampletable = Table.read("./../../../data/sampletable/phangs_sample_table_v1p6.fits")

mask = sampletable['name'] == galaxy
sampletable = sampletable[mask]

dist = sampletable['dist'].quantity[0]
pcperarcsec = dist.to('pc').value/206265
print('pcperarcsec ---> %0.001f' %pcperarcsec)

pcperarcsec ---> 47.7


In [4]:
# Get MUSE data for galaxy
muscat_file = '/Users/abarnes/Dropbox/work/Projects/pressures/phangs/data/catalouge/v2/physprops/master_notmasked.fits' #MUSE catalogue with properties

muscat_table = Table.read(muscat_file)
muscat_table = muscat_table[muscat_table['gal_name'] == galaxy.swapcase()]
muscat_table.rename_column('region_ID', 'muscat_id')

# Get MUSE REFITTED data for galaxy
muscat_newfits_file = '/Users/abarnes/Dropbox/work/Projects/pressures/phangs/data/catalouge/v2/raw/Nebulae_catalogue_v2_refitNII_refitTe.fits' #MUSE catalogue with properties

muscat_newfits_table = Table.read(muscat_newfits_file)
muscat_newfits_table = muscat_newfits_table[muscat_newfits_table['GAL_NAME'] == galaxy.swapcase()]
muscat_newfits_table.rename_column('REGION_ID', 'muscat_id')
muscat_newfits_table = muscat_newfits_table['muscat_id', 'T_N2_REFIT']
muscat_table = join(muscat_table, muscat_newfits_table, 'muscat_id')

In [5]:
hdus = load_pickle('../../analysis/catalogue/hdus_sample_sci.pickel')

[INFO] Load ../../analysis/catalogue/hdus_sample_sci.pickel


In [6]:
def get_regions(regions_file, hdu):
    
    regions = pyregion.open(regions_file).as_imagecoord(hdu.header)
    n = len(regions)
    
    ra = np.empty(n)*np.nan*au.deg
    dec = np.empty(n)*np.nan*au.deg
    width = np.empty(n)*np.nan*au.deg
    height = np.empty(n)*np.nan*au.deg
    position = ['']*n
    
    for i, region in enumerate(regions):

        ra[i] = float(region.params[0].text)*au.deg
        dec[i] = float(region.params[1].text)*au.deg
        width[i] = region.params[2].degree*au.deg
        height[i] = region.params[3].degree*au.deg
        
    position = SkyCoord(ra=ra, dec=dec, frame='icrs')

    return({'ra':ra, 'dec':dec, 'width':width, 'height':height, 'position': position})

regions_file = './../../analysis/catalogue/sample_v2.reg'
regions = get_regions(regions_file, hdus['hst07_hdus'][0])

In [7]:
def get_maskeddata(muscat_hdu, hst_hdu): 
    
    
    hst_masked_hdu = hst_hdu.copy() #only ID in mask
    hst_maskedall_hdu = hst_hdu.copy() #all MUSE regions out of mask - for noise measure
    
    #get mask 
    shape = muscat_hdu.data.shape
    muscat_id = muscat_hdu.data[int(shape[0]/2),int(shape[0]/2)]
    
    mask1 = muscat_hdu.data!=muscat_id #if catalouge is not ID
    mask2 = np.isnan(muscat_hdu.data) #if catalouge is not a number
    
    muscat_hdu1 = muscat_hdu.copy()
    muscat_hdu2 = muscat_hdu.copy()
    
    muscat_hdu1.data[mask1] = np.nan
    muscat_hdu2.data[mask2] = 1
    muscat_hdu2.data[~mask2] = np.nan

    #regrid
    data1 = reproject_interp(muscat_hdu1, hst_hdu.header, return_footprint=False, order='nearest-neighbor')
    data2 = reproject_interp(muscat_hdu2, hst_hdu.header, return_footprint=False, order='nearest-neighbor')

    # muscat_regrid_hdu = fits.PrimaryHDU(data, hst_hdu.header)

    #mask 
    hst_masked_hdu.data[np.isnan(data1)] = np.nan
    hst_maskedall_hdu.data[np.isnan(data2)] = np.nan
    
    return(hst_masked_hdu, hst_maskedall_hdu, muscat_id)

In [57]:
def get_dedro(hdu, hdu_outmask, hdu_nomask, min_npix=9, min_value_sig=3, min_delta_sig=3):
    """
    Perform dendrogram analysis on the input data and extract properties of the identified structures.
    
    Parameters:
        hdu (HDU): Input data HDU.
        hdu_outmask (HDU): Output mask HDU.
        hdu_nomask (HDU): Non-masked data HDU.
        min_npix (int, optional): Minimum number of pixels within a structure. Defaults to 9.
        min_value_sig (int, optional): Minimum value within a structure in terms of standard deviation. Defaults to 3.
        min_delta_sig (int, optional): Minimum values between levels within a structure in terms of standard deviation. Defaults to 3.
    
    Returns:
        tuple: A tuple containing the following:
            - dendro (Dendrogram): The computed dendrogram.
            - props (Table): Properties of the identified structure.
            - indexmap_trunk_hdu (HDU): Index map of the trunk structure.
            - indexmap_trunk_close_hdu (HDU): Index map of the trunk structure with closing operation applied.
    """

    # Replace variable names with appropriate names
    # hdu = hst06_masked_hdu
    wcs = WCS(hdu.header)
    header = hdu.header
    data = hdu.data
    data_nomask = hdu_nomask.data
    data_outmask = hdu_outmask.data

    # Calculate statistics for dendrogram
    std = stats.mad_std(data_outmask, ignore_nan=True)  # Get noise
    std = stats.mad_std(data_outmask[data_outmask<20*std], ignore_nan=True)  # Get noise below threshold

    pixsize = np.array([-1*hdu.header['CDELT1'], hdu.header['CDELT2']]).mean() * au.degree
    bmaj = bmin = 0.05 * au.arcsec  # Dummy values 

    min_value = std * min_value_sig  # Minimum value within structure
    min_delta = std * min_delta_sig  # Minimum values between levels within structure (sort of)

    # Running the dendrogram
    dendro = Dendrogram.compute(data,
                                min_delta=min_delta,
                                min_value=min_value,
                                min_npix=min_npix,
                                wcs=wcs)

    # Provide metadata for table output
    metadata = {}
    metadata['data_unit'] = au.Jy / au.beam  # Dummy unit
    metadata['spatial_scale'] = pixsize.to('arcsec')
    metadata['beam_major'] = bmaj.to('arcsec')
    metadata['beam_minor'] = bmin.to('arcsec')

    if len(np.unique(dendro.index_map)) != 1: 
        props = pp_catalog(dendro, metadata, verbose=False)  # Get table
    else: 
        return([None] * 4)

    indexmap = dendro.index_map  # Get index map
    indexmap_hdu = fits.PrimaryHDU(indexmap, header)
    indexmap_hdu.data = np.array(indexmap_hdu.data, dtype=float)
    indexmap_hdu.data[indexmap_hdu.data == -1] = np.nan

    indexmap_trunk = get_trunkindexmap(dendro)
    indexmap_trunk_hdu = fits.PrimaryHDU(indexmap_trunk, header)
    indexmap_trunk_hdu.data = np.array(indexmap_trunk_hdu.data, dtype=float)

    props_trunk = pp_catalog(dendro.trunk, metadata, verbose=False)  # Get table
    props_trunk_ = pp_catalog(dendro.trunk, metadata, verbose=False)  # Get table

    # Get the ID of the structure with maximum flux
    arg = np.argmax(props_trunk['flux'])
    midpix_id = props_trunk['_idx'][arg]

    # Remove all other structures from the catalogue
    indexmap_trunk_hdu.data[indexmap_trunk_hdu.data != midpix_id] = -1
    props = props[props['_idx'] == midpix_id]

    # Get single trunk index map
    idx = np.unique(indexmap_trunk_hdu.data[indexmap_trunk_hdu.data != -1])
    mask = indexmap_trunk_hdu.data != -1
    mask1 = scipy.ndimage.binary_closing(mask, iterations=10) * 1
    mask1[mask1 != 1.0] = -1.
    mask1[mask1 == 1.0] = idx
    indexmap_trunk_close = np.copy(mask1)
    indexmap_trunk_close_hdu = fits.PrimaryHDU(indexmap_trunk_close, indexmap_trunk_hdu.header)

    radius_trunk = get_circrad(indexmap_trunk_hdu.data, pixsize)
    radius_trunkclose = get_circrad(indexmap_trunk_close_hdu.data, pixsize)

    # Add properties to table    
    props['radius_trunk'] = radius_trunk
    props['radius_trunkclose'] = radius_trunkclose
    props['major_fwtm'] = props['major_sigma'] * 4.292
    props['minor_fwtm'] = props['minor_sigma'] * 4.292
    props['mean_fwtm'] = np.nanmean([props['major_fwtm'], props['minor_fwtm']]) * props['major_fwtm'].unit
    props['mean_hwtm'] = props['mean_fwtm'] / 2.
    
    props['min_npix'] = min_npix
    props['min_value_sig'] = min_value_sig
    props['min_delta_sig'] = min_delta_sig
    
    # Convert to parcsec
    props['radius_trunk_pc'] = props['radius_trunk'].quantity[0].to('arcsec').value * pcperarcsec * au.pc
    props['radius_trunkclose_pc'] = props['radius_trunkclose'].quantity[0].to('arcsec').value * pcperarcsec * au.pc
    props['major_fwtm_pc'] = props['major_fwtm'].quantity[0].to('arcsec').value * pcperarcsec * au.pc
    props['minor_fwtm_pc'] = props['minor_fwtm'].quantity[0].to('arcsec').value * pcperarcsec * au.pc
    props['mean_fwtm_pc'] = props['mean_fwtm'].quantity[0].to('arcsec').value * pcperarcsec * au.pc
    props['mean_hwtm_pc'] = props['mean_fwtm_pc'] / 2.

    ax = aplpy.FITSFigure(hdu)
    ra_struc = ax.pixel2world(props['x_cen'], props['y_cen'])[0][0]
    dec_struc = ax.pixel2world(props['x_cen'], props['y_cen'])[1][0]
    plt.close('all')

    props['ra_cent'] = ra_struc * au.deg
    props['dec_cent'] = dec_struc * au.deg
    props.rename_column('radius', 'mean_sigma')
    props['mean_sigma_pc'] = props['mean_sigma'].quantity[0].to('arcsec').value * pcperarcsec * au.pc

    flux = np.nansum(hdu.data[indexmap_trunk_close_hdu.data != -1])
    props['flux'] = flux
    props['flux'].unit = au.erg / au.s / au.cm**2

    return dendro, props, indexmap_trunk_hdu, indexmap_trunk_close_hdu

In [99]:
import concurrent.futures
from tqdm.auto import tqdm

def process_regions(hdus, regions, hstha_hdu_name='hst07_hdus'):
    # Initialize lists to store the processed data
    hstha_hdu_smooth = []
    hstha_masked_hdu = []
    hstha_maskedall_hdu = []
    muscat_id = []
    mask_muse = []

    def process_region(regionID):
        # Load the necessary data
        muscat_hdu = hdus['muscat_hdus'][regionID]
        hstha_hdu = hdus[hstha_hdu_name][regionID].copy()

        # Smooth the data
        kernel = Gaussian2DKernel(x_stddev=0.5)
        hstha_hdu.data = convolve(hstha_hdu.data, kernel)

        # Mask the data
        hstha_masked_hdu, hstha_maskedall_hdu, muscat_id = get_maskeddata(muscat_hdu, hstha_hdu)
        mask_muse = fits.PrimaryHDU(~np.isnan(hstha_masked_hdu.data) * 1, hstha_hdu.header)

        # Return the processed data along with the regionID
        return regionID, hstha_hdu, hstha_masked_hdu, hstha_maskedall_hdu, muscat_id, mask_muse

    for regionID in tqdm(range(len(regions['ra'])), desc='Masking regions', position=0):

        # Extract the results in the correct order
        results = process_region(regionID)
        hstha_hdu_smooth += [results[1]]
        hstha_masked_hdu += [results[2]]
        hstha_maskedall_hdu += [results[3]]
        muscat_id += [results[4]]
        mask_muse += [results[5]]

    # Assign the processed data to the corresponding keys in the hdus dictionary
    hdus['%s_smooth' % hstha_hdu_name] = hstha_hdu_smooth
    hdus['%s_smooth_masked' % hstha_hdu_name] = hstha_masked_hdu
    hdus['%s_smooth_maskedall' % hstha_hdu_name] = hstha_maskedall_hdu
    hdus['musmask_hdus'] = mask_muse

    # Return the modified hdus dictionary and muscat_id
    return hdus, muscat_id

# Call the process_regions function and store the returned values
hdus, muscat_id = process_regions(hdus, regions)

Masking regions:   0%|          | 0/215 [00:00<?, ?it/s]

In [10]:
def get_scaling(data):
    data = (data-np.nanmin(data))/(np.nanmax(data)-np.nanmin(data))
    return(data)

def make_plot_sigmacomp(hdu, props, indexmap_trunk_hdu, regions, regionID, mask_muse, min_value_sig, outdir='./../../analysis/dendro/indexmaps_figs_sigmacomp/'):

    """Make plot"""
    
    ra = regions['ra'][regionID].value
    dec = regions['dec'][regionID].value
    width = regions['width'][regionID].value
    height = regions['height'][regionID].value
    radius = max([width,height])
    center = [ra, dec, radius, radius]
    
    fig = plt.figure(figsize=(10,10))
    ax = ['']*3
    
    hdu_ = hdu.copy()
    hdu_.data = get_scaling(hdu_.data)
    
    for i in range(3):
                
        ax[i] = aplpy.FITSFigure(hdu_, figure=fig, subplot=(1,3,i+1))
        minmax = np.nanpercentile(hdu_.data, [0.1,99.99])
        ax[i].show_colorscale(cmap='gist_gray_r', vmin=minmax[0], vmax=minmax[-1], stretch='log')
        ax[i].recenter(center[0], center[1], width=center[2], height=center[3])
        ax[i].tick_labels.hide()
        ax[i].axis_labels.hide()
        ax[i].ticks.set_color('black')
        ax[i].set_nan_color('white')

        if props[i]==None: 
            ax[i].add_label(0.5,0.5,'NO FIT',c='r',relative=True)
            continue
        
        ra_struc = ax[i].pixel2world(props[i]['x_cen'], props[i]['y_cen'])[0][0]
        dec_struc = ax[i].pixel2world(props[i]['x_cen'], props[i]['y_cen'])[1][0]   
    
        ax[i].show_contour(indexmap_trunk_hdu[i], colors='C1', levels=[-1], linestyles='-')
    
        ax[i].show_ellipses(xw=ra_struc, yw=dec_struc, 
                 width=props[i]['radius_trunk'].quantity.to('deg').value[0]*2,
                 height=props[i]['radius_trunk'].quantity.to('deg').value[0]*2,
                 linestyle='--', edgecolor='C1', linewidth=2)
    
        #Ellipses
        ax[i].show_ellipses(xw=ra_struc, yw=dec_struc, 
                         width=props[i]['major_sigma'].quantity.to('deg').value[0]*2,#sigma
                         height=props[i]['minor_sigma'].quantity.to('deg').value[0]*2,#sigma
                         angle=props[i]['position_angle'].quantity.to('deg').value[0],
                         linestyle='-', edgecolor='C3', linewidth=2)

        ax[i].show_ellipses(xw=ra_struc, yw=dec_struc, 
                         width=props[i]['major_sigma'].quantity.to('deg').value[0]*4.292,#FWTM
                         height=props[i]['minor_sigma'].quantity.to('deg').value[0]*4.292,#FWTM
                         angle=props[i]['position_angle'].quantity.to('deg').value[0],
                         linestyle='--', edgecolor='C3', linewidth=2)  

        #Mean of ellipses
        ax[i].show_ellipses(xw=ra_struc, yw=dec_struc, 
                         width=props[i]['mean_sigma'].quantity.to('deg').value[0]*2,#sigma
                         height=props[i]['mean_sigma'].quantity.to('deg').value[0]*2,#sigma
                         linestyle=':', edgecolor='C3', linewidth=2)

        ax[i].show_ellipses(xw=ra_struc, yw=dec_struc, 
                         width=props[i]['mean_sigma'].quantity.to('deg').value[0]*4.292,#FWTM
                         height=props[i]['mean_sigma'].quantity.to('deg').value[0]*4.292,#FWTM
                         linestyle=':', edgecolor='C3', linewidth=2) 

        ax[i].add_label(0.02,0.95,'min_value_sig=%isigma' %min_value_sig[i],relative=True,ha='left')
        
    if mask_muse!=None:
        for i in range(3):
            ax[i].show_contour(mask_muse, colors='black', levels=[1], linestyles='-')
            
        fig.get_axes()[0].plot([0,0],[0,0],ls='-',c='black',label='MUSE mask')
        
    
    fig.tight_layout()
    fig.savefig('%s/dendro_regionoutput_%i.pdf' %(outdir, regionID), bbox_inches='tight', dpi=300)
    plt.close('all')
    
    return()

In [172]:
def get_dedro_all(regionID):
    
    # Initialize variables for storing the results
    props = ['']*min_value_sig_n
    indexmap_trunk_hdu = ['']*min_value_sig_n
    indexmap_trunk_close_hdu = ['']*min_value_sig_n

    # Process each sigma value
    for i in range(min_value_sig_n):
        
        output = get_dedro(hdus['hst07_hdus_smooth_masked'][regionID],
                           hdus['hst07_hdus_smooth_maskedall'][regionID], 
                           hdus['hst07_hdus_smooth'][regionID],
                           min_npix=9, 
                           min_value_sig=min_value_sig[i], 
                           min_delta_sig=3)

        dendro, props[i], indexmap_trunk_hdu[i], indexmap_trunk_close_hdu[i] = output
    
    # Add more info for each sigma value
    for i in range(min_value_sig_n):
        
        # Replace values
        if props[i] is None: 
            
            print(f'[INFO] [get_dedro] [regionID={regionID}] [min_value_sig={min_value_sig[i]}] No dendro found, masking all values in table...')
            props[i] = get_dedro(hdus['hst07_hdus_smooth_masked'][regionID],
                                hdus['hst07_hdus_smooth_maskedall'][regionID], 
                                hdus['hst07_hdus_smooth'][regionID],
                                min_npix=0, 
                                min_value_sig=-100, 
                                min_delta_sig=-100)[1]
                
            for colname in props[i].colnames: 
                props[i][colname] = np.ma.masked
        
        props[i]['id'] = regionID
        props[i]['muscat_id'] = muscat_id[regionID]

    # Return the results along with the regionID
    return regionID, props, indexmap_trunk_hdu

min_value_sig = [2, 3, 4, 5]
min_value_sig_n = len(min_value_sig)
min_value_sig_str = ['%ssig' %min_value_sig[i] for i in range(min_value_sig_n)]
props_all = dict.fromkeys(min_value_sig_str)
indexmap_trunk_hdus = hdus['indexmap_trunk_hdus'] = []

for regionID in tqdm(range(len(regions['ra'])), desc='Dendrogram (with sigma ranges)', position=0):
    
    if all_nan_check(hdus, regionID):
        continue
    
    regionID, props, indexmap_trunk_hdu = get_dedro_all(regionID)
    
    try: 
        for i in range(min_value_sig_n): 
            props_all['%isig'%min_value_sig[i]].add_row(props[i][0])
    except: 
        for i in range(min_value_sig_n): 
            props_all['%isig'%min_value_sig[i]] = props[i].copy()        
    try: 
        indexmap_trunk_hdus.append(indexmap_trunk_hdu[1].copy())
    except: 
        print(f'[INFO] [props_all] [regionID={regionID}] No 3 sigma mask, all blank added to indexmap_trunk_hdu...')
        shape = hdus['hst07_hdus_smooth_masked'][regionID].data.shape
        header = hdus['hst07_hdus_smooth_masked'][regionID].header
        indexmap_trunk_hdus.append(fits.PrimaryHDU(np.ones(shape)*-1, header))

# Assign the indexmap_trunk_hdus to hdus dictionary
hdus['indexmap_trunk_hdus'] = indexmap_trunk_hdus  # only taking the 3sigma

Dendrogram (with sigma ranges):   0%|          | 0/215 [00:00<?, ?it/s]

[INFO] [get_dedro] [regionID=20] [min_value_sig=4] No dendro found, masking all values in table...
[INFO] [get_dedro] [regionID=20] [min_value_sig=5] No dendro found, masking all values in table...
[INFO] [get_dedro] [regionID=44] [min_value_sig=5] No dendro found, masking all values in table...
[INFO] [get_dedro] [regionID=63] [min_value_sig=3] No dendro found, masking all values in table...
[INFO] [get_dedro] [regionID=63] [min_value_sig=4] No dendro found, masking all values in table...
[INFO] [get_dedro] [regionID=63] [min_value_sig=5] No dendro found, masking all values in table...
[INFO] [props_all] [regionID=63] No 3 sigma mask, all blank added to indexmap_trunk_hdu...
[INFO] [get_dedro] [regionID=81] [min_value_sig=4] No dendro found, masking all values in table...
[INFO] [get_dedro] [regionID=81] [min_value_sig=5] No dendro found, masking all values in table...
[INFO] [get_dedro] [regionID=100] [min_value_sig=5] No dendro found, masking all values in table...
[INFO] [get_dedro

In [173]:
def all_nan_check(hdus, regionID):
    
    data = hdus['hst07_hdus_smooth'][regionID].data
    shape = data.shape
    xmin = shape[0]/2 - shape[0]/4
    xmax = shape[0]/2 + shape[0]/4
    ymin = shape[1]/2 - shape[1]/4
    ymax = shape[1]/2 + shape[1]/4
    
    data = data[int(xmin):int(xmax),int(ymin):int(ymax)]
    
    if np.isnan(data).all():
        print(f'[INFO] [props_all] [regionID={regionID}] All nan values in map, skipping...')
        return (True)
    else:
        return (False)

In [174]:
# Add in MUSE informaiton
for i in range(min_value_sig_n): 
    props_all['%isig'%min_value_sig[i]] = join(props_all['%isig'%min_value_sig[i]], muscat_table, 'muscat_id')

In [177]:
save_pickle(hdus, '../../analysis/catalogue/hdus_sample_withdendro.pickel') 
save_pickle(props_all, '../../analysis/dendro/props_all/01_props_all_allsig.pickel') 

for i in range(min_value_sig_n): 
    filename = '../../analysis/dendro/props_all/01_props_all_%isig.fits' %min_value_sig[i]
    print('[INFO] [save_fits] Saved to %s' %filename)
    props_all['%isig' %min_value_sig[i]].write(filename, overwrite=True)

[INFO] Saved to ../../analysis/catalogue/hdus_sample_withdendro.pickel
[INFO] Saved to ../../analysis/dendro/props_all/01_props_all_allsig.pickel
[INFO] [save_fits] Saved to ../../analysis/dendro/props_all/01_props_all_2sig.fits
[INFO] [save_fits] Saved to ../../analysis/dendro/props_all/01_props_all_3sig.fits
[INFO] [save_fits] Saved to ../../analysis/dendro/props_all/01_props_all_4sig.fits
[INFO] [save_fits] Saved to ../../analysis/dendro/props_all/01_props_all_5sig.fits
